# Installation and Imports

In [ ]:
pip install tensorflow numpy pandas sklearn matplotlib nltk

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('punkt')


# Loading the Dataset

In [ ]:
# Sample code assumes a CSV file with columns "review" and "sentiment"
dataset_path = 'path/to/your/dataset.csv'  # Provide the path to your dataset

# Load the dataset
data = pd.read_csv(dataset_path)

# Check the first few rows of the dataset
print(data.head())


# Data Preprocessing

In [ ]:
# Preprocess the dataset
MAX_LEN = 100  # Maximum length of sequences
VOCAB_SIZE = 10000  # Size of the vocabulary

# Tokenize and pad the sequences
tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token='<OOV>')
tokenizer.fit_on_texts(data['review'])

# Convert the text to sequences
sequences = tokenizer.texts_to_sequences(data['review'])

# Pad the sequences to the same length
padded_sequences = pad_sequences(sequences, maxlen=MAX_LEN, padding='post', truncating='post')

# Convert sentiments to numerical labels
sentiment_to_index = {'positive': 1, 'negative': 0, 'neutral': 2}
data['sentiment_index'] = data['sentiment'].map(sentiment_to_index)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, data['sentiment_index'], test_size=0.2, random_state=42)


# Model Preparation and Training

In [ ]:
# Define the model
model = Sequential([
    Embedding(input_dim=VOCAB_SIZE, output_dim=128, input_length=MAX_LEN),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


# Testing the Model

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

# Generate a classification report
y_pred = model.predict_classes(X_test)
print(classification_report(y_test, y_pred, target_names=['negative', 'positive', 'neutral']))


# Saving and Loading the Model

In [ ]:
# Save the model
model.save('sentiment_analysis_model.h5')

# Load the model
loaded_model = tf.keras.models.load_model('sentiment_analysis_model.h5')


# Performing Sentiment Analysis on New Reviews

In [ ]:
def perform_sentiment_analysis(review, tokenizer, model):
    # Tokenize and pad the review
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=MAX_LEN, padding='post', truncating='post')

    # Predict the sentiment
    prediction = model.predict(padded_sequence)
    sentiment_index = np.argmax(prediction)
    index_to_sentiment = {0: 'negative', 1: 'positive', 2: 'neutral'}

    # Return the predicted sentiment
    return index_to_sentiment[sentiment_index]

# Example usage
new_review = "The product quality is great, but the delivery was late."
predicted_sentiment = perform_sentiment_analysis(new_review, tokenizer, model)
print(f'Predicted Sentiment: {predicted_sentiment}')
